In [1]:
#import dependencies
import pandas as pd
from pathlib import Path
import numpy as np

#matplotlib
%matplotlib inline
import matplotlib.pyplot as plt

In [33]:
#load business into df
business_path = Path('data_zipped/yelp_business.csv.zip')
business_df = pd.read_csv(business_path)
business_df.head()

,business_id,name,neighborhood,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories
0,FYWN1wneV18bWNgQjJ2GNg,"""Dental by Design""",NaN,"""4855 E Warner Rd, Ste B9""",Ahwatukee,AZ,85044,33.330690,-111.978599,4.0,22,1,Dentists;General Dentistry;Health & Medical;Or...
1,He-G7vWjzVUysIKrfNbPUQ,"""Stephen Szabo Salon""",NaN,"""3101 Washington Rd""",McMurray,PA,15317,40.291685,-80.104900,3.0,11,1,Hair Stylists;Hair Salons;Men's Hair Salons;Bl...
2,KQPW8lFf1y5BT2MxiSZ3QA,"""Western Motor Vehicle""",NaN,"""6025 N 27th Ave, Ste 1""",Phoenix,AZ,85017,33.524903,-112.115310,1.5,18,1,Departments of Motor Vehicles;Public Services ...
3,8DShNS-LuFqpEWIp0HxijA,"""Sports Authority""",NaN,"""5000 Arizona Mills Cr, Ste 435""",Tempe,AZ,85282,33.383147,-111.964725,3.0,9,0,Sporting Goods;Shopping
4,PfOCPjBrlQAnz__NXj9h_w,"""Brick House Tavern + Tap""",NaN,"""581 Howe Ave""",Cuyahoga Falls,OH,44221,41.119535,-81.475690,3.5,116,1,American (New);Nightlife;Bars;Sandwiches;Ameri...


In [34]:
business_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 174567 entries, 0 to 174566
Data columns (total 13 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   business_id   174567 non-null  object 
 1   name          174567 non-null  object 
 2   neighborhood  68015 non-null   object 
 3   address       174567 non-null  object 
 4   city          174566 non-null  object 
 5   state         174566 non-null  object 
 6   postal_code   173944 non-null  object 
 7   latitude      174566 non-null  float64
 8   longitude     174566 non-null  float64
 9   stars         174567 non-null  float64
 10  review_count  174567 non-null  int64  
 11  is_open       174567 non-null  int64  
 12  categories    174567 non-null  object 
dtypes: float64(3), int64(2), object(8)
memory usage: 17.3+ MB


In [35]:
business_df.describe()

,latitude,longitude,stars,review_count,is_open
count,174566.000000,174566.000000,174567.000000,174567.000000,174567.000000
mean,38.627312,-92.679009,3.632196,30.137059,0.840376
std,5.389012,26.240079,1.003739,98.208174,0.366258
min,-36.086009,-142.466650,1.000000,3.000000,0.000000
25%,33.631550,-112.125879,3.000000,4.000000,1.000000
50%,36.144257,-89.410128,3.500000,8.000000,1.000000
75%,43.606181,-79.657609,4.500000,23.000000,1.000000
max,89.999314,115.086769,5.000000,7361.000000,1.000000


In [36]:
categories_df = business_df[['business_id', 'categories']].copy()
# categories_df = business_df[['business_id', "name", "state",'categories']].copy()
categories_df['categories'] = business_df['categories'].str.split(';', expand=False)
categories_df.head()

,business_id,categories
0,FYWN1wneV18bWNgQjJ2GNg,"[Dentists, General Dentistry, Health & Medical..."
1,He-G7vWjzVUysIKrfNbPUQ,"[Hair Stylists, Hair Salons, Men's Hair Salons..."
2,KQPW8lFf1y5BT2MxiSZ3QA,"[Departments of Motor Vehicles, Public Service..."
3,8DShNS-LuFqpEWIp0HxijA,"[Sporting Goods, Shopping]"
4,PfOCPjBrlQAnz__NXj9h_w,"[American (New), Nightlife, Bars, Sandwiches, ..."


In [37]:
categories_df["is_restaurant"] = np.where(business_df.categories.str.contains("Restaurant"),1,0)
# categories_df["is_mexican_restaurant"] = np.where(business_df.categories.str.contains("Mex"),1,0)

# categories_df["is_restaurant"] = np.where((business_df.categories.str.contains("Restaurant")) | (business_df.categories.str.contains("Food")),1,0)
# categories_df["is_mexican_restaurant"] = np.where((business_df.categories.str.contains("Mex")) | (business_df.categories.str.contains("Latin")),1,0)

restaurants_df = categories_df[categories_df.is_restaurant == 1]
# restaurants_df.head()

In [38]:
# use explode to convert list elements to rows
categories_pivot = restaurants_df.assign(category=restaurants_df.categories.str.split(",")).explode('categories')
categories_pivot.loc[:, 'business_id':'is_restaurant']
categories_pivot.head()

,business_id,categories,is_restaurant,category
4,PfOCPjBrlQAnz__NXj9h_w,American (New),1,NaN
4,PfOCPjBrlQAnz__NXj9h_w,Nightlife,1,NaN
4,PfOCPjBrlQAnz__NXj9h_w,Bars,1,NaN
4,PfOCPjBrlQAnz__NXj9h_w,Sandwiches,1,NaN
4,PfOCPjBrlQAnz__NXj9h_w,American (Traditional),1,NaN


In [ ]:
len(categories_pivot.categories.unique())
# with pd.option_context('display.max_rows', None,):
#     print(categories_pivot.categories.nunique())

In [39]:
# categories_pivot = categories_pivot.set_index(['business_id', 'name', "state","categories"], drop=True).unstack("categories")
categories_pivot = categories_pivot.set_index(['business_id', "categories"], drop=True).unstack("categories")
categories_pivot = categories_pivot.replace(np.nan, 0)

categories_pivot.head()


is_restaurant                                     \
categories                & Probates Acai Bowls Accessories Accountants   
business_id                                                               
--6MefnULPED_I942VcFNA           0.0        0.0         0.0         0.0   
--9e1ONYQuAa-CB_Rrw7Tw           0.0        0.0         0.0         0.0   
--DaPTJW3-tB1vP-PfdTEg           0.0        0.0         0.0         0.0   
--FBCX-N37CMYDfs790Bnw           0.0        0.0         0.0         0.0   
--GM_ORV2cYS-h38DSaCLw           0.0        0.0         0.0         0.0   

                                                                      \
categories             Active Life Acupuncture Adult Adult Education   
business_id                                                            
--6MefnULPED_I942VcFNA         0.0         0.0   0.0             0.0   
--9e1ONYQuAa-CB_Rrw7Tw         0.0         0.0   0.0             0.0   
--DaPTJW3-tB1vP-PfdTEg         0.0         0.0   0.0             0.0   
--FBCX-N37CMYDfs790Bnw         0.0         0.0   0.0             0.0   
--GM_ORV2cYS-h38DSaCLw         0.0         0.0   0.0             0.0   

                                                        ...  category  \
categories             Adult Entertainment Advertising  ... Wine Bars   
business_id                                             ...             
--6MefnULPED_I942VcFNA                 0.0         0.0  ...       0.0   
--9e1ONYQuAa-CB_Rrw7Tw                 0.0         0.0  ...       0.0   
--DaPTJW3-tB1vP-PfdTEg                 0.0         0.0  ...       0.0   
--FBCX-N37CMYDfs790Bnw                 0.0         0.0  ...       0.0   
--GM_ORV2cYS-h38DSaCLw                 0.0         0.0  ...       0.0   

                                                                   \
categories             Wine Tasting Room Wine Tours Wineries  Wok   
business_id                                                         
--6MefnULPED_I942VcFNA               0.0        0.0      0.0  0.0   
--9e1ONYQuAa-CB_Rrw7Tw               0.0        0.0      0.0  0.0   
--DaPTJW3-tB1vP-PfdTEg               0.0        0.0      0.0  0.0   
--FBCX-N37CMYDfs790Bnw               0.0        0.0      0.0  0.0   
--GM_ORV2cYS-h38DSaCLw               0.0        0.0      0.0  0.0   

                                                                     
categories             Women's Clothing Wraps Yelp Events Yoga Zoos  
business_id                                                          
--6MefnULPED_I942VcFNA              0.0   0.0         0.0  0.0  0.0  
--9e1ONYQuAa-CB_Rrw7Tw              0.0   0.0         0.0  0.0  0.0  
--DaPTJW3-tB1vP-PfdTEg              0.0   0.0         0.0  0.0  0.0  
--FBCX-N37CMYDfs790Bnw              0.0   0.0         0.0  0.0  0.0  
--GM_ORV2cYS-h38DSaCLw              0.0   0.0         0.0  0.0  0.0  

[5 rows x 1396 columns]

In [40]:
categories_pivot = pd.merge(business_df, categories_pivot, on='business_id', how='inner')
categories_pivot.head()

C:\Users\curt0\AppData\Local\Temp/ipykernel_11812/268916982.py:1: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left,2 on the right)
  categories_pivot = pd.merge(business_df, categories_pivot, on='business_id', how='inner')


,business_id,name,neighborhood,address,city,state,postal_code,latitude,longitude,stars,...,"(category, Wine Bars)","(category, Wine Tasting Room)","(category, Wine Tours)","(category, Wineries)","(category, Wok)","(category, Women's Clothing)","(category, Wraps)","(category, Yelp Events)","(category, Yoga)","(category, Zoos)"
0,PfOCPjBrlQAnz__NXj9h_w,"""Brick House Tavern + Tap""",NaN,"""581 Howe Ave""",Cuyahoga Falls,OH,44221,41.119535,-81.475690,3.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,o9eMRCWt5PkpLDE0gOPtcQ,"""Messina""",NaN,"""Richterstr. 11""",Stuttgart,BW,70567,48.727200,9.147950,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,XOSRcvtaKc_Q5H1SAzN20A,"""East Coast Coffee""",NaN,"""737 West Pike St""",Houston,PA,15342,40.241548,-80.212815,4.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,fNMVV_ZX7CJSDWQGdOM8Nw,"""Showmars Government Center""",Uptown,"""600 E 4th St""",Charlotte,NC,28202,35.221647,-80.839345,3.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,l09JfMeQ6ynYs5MCJtrcmQ,"""Alize Catering""",Yonge and Eglinton,"""2459 Yonge St""",Toronto,ON,M4P 2H6,43.711399,-79.399339,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [42]:
with pd.option_context('display.max_rows', None,):
    print(categories_pivot.nunique())

categories_pivot.to_csv('data/categories_pivot.csv')

business_id                                          54630
name                                                 38163
neighborhood                                           378
address                                              44880
city                                                   782
state                                                   45
postal_code                                           9769
latitude                                             48654
longitude                                            48780
stars                                                    9
review_count                                           991
is_open                                                  2
categories                                           25774
(is_restaurant, & Probates)                              2
(is_restaurant, Acai Bowls)                              2
(is_restaurant, Accessories)                             2
(is_restaurant, Accountants)                            

In [ ]:
#load attributes into df
attributes_path = Path('data_zipped/yelp_business_attributes.csv.zip')
attributes_df = pd.read_csv(attributes_path)
attributes_df.head()

In [ ]:
attributes_df.info()

In [ ]:
attributes_df.describe()

In [ ]:
# Check unique value counts
with pd.option_context('display.max_rows', None,):
    print(business_df.nunique())

In [ ]:
# Odd...
business_df.HairSpecializesIn_coloring.value_counts()

In [ ]:
# Ah, resorts!
business_df[business_df["HairSpecializesIn_coloring"] == "True"]

In [ ]:
#load tips into df
tips_path = Path('data_zipped/yelp_tip.csv.zip')
tips_df = pd.read_csv(tips_path)
tips_df.head()

In [ ]:
tips_df.info()

In [ ]:
tips_df.describe()